# Notebook 01: Addestramento e Valutazione dei Modelli

**Scopo:** Questo notebook carica i dati pre-processati dal Notebook 00, definisce le architetture delle reti neurali, orchestra un ciclo di esperimenti per addestrare e valutare diverse combinazioni di modelli e ottimizzatori, e salva gli artefatti migliori per l'analisi successiva.

**Input:**
- Dati pre-processati da `../data/processed/` (`X_train.npy`, `y_train.npy`, etc.)

**Output (salvati in `../models/` e `../reports/`):**
- I modelli migliori per ogni esperimento (es. `UNet_Lite_Adam.keras`).
- Un file di riepilogo con le metriche di performance (es. `training_summary.csv`).
- (Opzionale) Le storie di training salvate.

In [1]:
# ===================================================================
# CELLA 1: SETUP, IMPORTS E CARICAMENTO DATI
# ===================================================================

import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import time
import traceback

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers
from tensorflow.keras.utils import to_categorical

# --- Configurazione Globale ---
PROCESSED_DATA_PATH = '../data/processed/'
MODELS_PATH = '../models/'
REPORTS_PATH = '../reports/'
RANDOM_STATE = 42

os.makedirs(MODELS_PATH, exist_ok=True)
os.makedirs(REPORTS_PATH, exist_ok=True)

# 1. GPU e Mixed Precision
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU(s) Trovata/e: {[tf.config.experimental.get_device_details(g)['device_name'] for g in gpus]}")
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        tf.keras.mixed_precision.set_global_policy(policy)
        print(f"✅ Politica di Mixed Precision impostata su: {tf.keras.mixed_precision.global_policy().name}")
    except RuntimeError as e: print(f"⚠️ Errore durante l'inizializzazione della GPU: {e}")
else: print("❌ NESSUNA GPU TROVATA. L'allenamento sarà su CPU.")

# 2. Caricamento Dati Pre-processati
print("\n🔄 Caricamento dei dati pre-processati...")
try:
    X_train = np.load(os.path.join(PROCESSED_DATA_PATH, 'X_train.npy'))
    y_train = np.load(os.path.join(PROCESSED_DATA_PATH, 'y_train.npy'))
    X_val = np.load(os.path.join(PROCESSED_DATA_PATH, 'X_val.npy'))
    y_val = np.load(os.path.join(PROCESSED_DATA_PATH, 'y_val.npy'))
    X_test = np.load(os.path.join(PROCESSED_DATA_PATH, 'X_test.npy'))
    y_test = np.load(os.path.join(PROCESSED_DATA_PATH, 'y_test.npy'))
    
    with open(os.path.join(PROCESSED_DATA_PATH, 'label_encoder.pkl'), 'rb') as f:
        label_encoder = pickle.load(f)

    # Conversione in formato categorico
    num_classes = len(label_encoder.classes_)
    y_train_cat = to_categorical(y_train, num_classes=num_classes)
    y_val_cat = to_categorical(y_val, num_classes=num_classes)
    y_test_cat = to_categorical(y_test, num_classes=num_classes)
    
    print("\n✅ Dati caricati con successo.")
    print(f"   - Shape X_train: {X_train.shape} | Shape y_train_cat: {y_train_cat.shape}")
    print(f"   - Numero di classi: {num_classes}")
except FileNotFoundError:
    print("❌ ERRORE: File di dati non trovati. Eseguire prima il notebook '00_Setup_and_Data_Preparation.ipynb'.")

2025-07-14 00:54:22.325877: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-14 00:54:22.326579: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-14 00:54:22.329349: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-14 00:54:22.339656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752447262.355529   18326 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752447262.35

❌ NESSUNA GPU TROVATA. L'allenamento sarà su CPU.

🔄 Caricamento dei dati pre-processati...


2025-07-14 00:54:23.835523: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)



✅ Dati caricati con successo.
   - Shape X_train: (3000, 128, 256, 1) | Shape y_train_cat: (3000, 10)
   - Numero di classi: 10


In [ ]:
# ===================================================================
# CELLA 2: DEFINIZIONE DELLE ARCHITETTURE DEI MODELLI
# ===================================================================

class ModelFactory:
    """Contiene i metodi statici per costruire le nostre architetture di modelli."""
    
    @staticmethod
    def _conv_block(inputs, num_filters, l2_reg_factor=0.001):
        l2_reg = regularizers.l2(l2_reg_factor)
        x = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_regularizer=l2_reg)(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.PReLU(shared_axes=[1, 2])(x)
        x = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_regularizer=l2_reg)(x)
        x = layers.BatchNormalization()(x)
        x = layers.PReLU(shared_axes=[1, 2])(x)
        return x

    @staticmethod
    def build_unet_lite_model(input_shape, num_classes):
        inputs = layers.Input(shape=input_shape)
        x = inputs
        h, w = input_shape[0], input_shape[1]
        pad_h = (16 - h % 16) % 16
        pad_w = (16 - w % 16) % 16
        if pad_h > 0 or pad_w > 0:
            x = layers.ZeroPadding2D(padding=((0, pad_h), (0, pad_w)))(x)
            
        # Encoder
        conv1 = ModelFactory._conv_block(x, 32)
        pool1 = layers.MaxPooling2D((2, 2))(conv1)
        conv2 = ModelFactory._conv_block(pool1, 64)
        pool2 = layers.MaxPooling2D((2, 2))(conv2)
        conv3 = ModelFactory._conv_block(pool2, 128)
        pool3 = layers.MaxPooling2D((2, 2))(conv3)
        conv4 = ModelFactory._conv_block(pool3, 256)
        pool4 = layers.MaxPooling2D((2, 2))(conv4)
        
        # Bottleneck
        bottleneck = ModelFactory._conv_block(pool4, 512)
        
        # Decoder
        up6 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(bottleneck)
        conv6 = ModelFactory._conv_block(layers.concatenate([conv4, up6]), 256)
        up7 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6)
        conv7 = ModelFactory._conv_block(layers.concatenate([conv3, up7]), 128)
        up8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7)
        conv8 = ModelFactory._conv_block(layers.concatenate([conv2, up8]), 64)
        up9 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8)
        final_conv = ModelFactory._conv_block(layers.concatenate([conv1, up9]), 32)

        if pad_h > 0 or pad_w > 0:
            final_conv = layers.Cropping2D(cropping=((0, pad_h), (0, pad_w)))(final_conv)

        # Head
        gap = layers.GlobalAveragePooling2D()(final_conv)
        x = layers.Dense(256, activation='relu')(gap)
        x = layers.Dropout(0.5)(x)
        outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
        
        return models.Model(inputs=inputs, outputs=outputs)

    @staticmethod
    def build_simple_cnn(input_shape, num_classes):
        inputs = layers.Input(shape=input_shape)
        
        x = layers.Conv2D(32, (3, 3), padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        
        x = layers.Conv2D(64, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        
        x = layers.Conv2D(128, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
        
        return models.Model(inputs=inputs, outputs=outputs)

print("✅ ModelFactory definita con i modelli 'UNet_Lite' e 'SimpleCNN'.")

✅ ModelFactory definita con i modelli 'UNet_Lite' e 'SimpleCNN'.


In [ ]:
# ===================================================================
# CELLA 3: FRAMEWORK DI TRAINING, PIPELINE DATI E ESECUZIONE
# ===================================================================

def spec_augment(spectrogram, label):
    # Funzione di augmentation (identica a prima)
    spectrogram_augmented = tf.identity(spectrogram)
    if tf.rank(spectrogram_augmented) == 3 and spectrogram_augmented.shape[-1] == 1:
        spectrogram_augmented = tf.squeeze(spectrogram_augmented, axis=-1)
    freq_bins, time_steps = tf.shape(spectrogram_augmented)[0], tf.shape(spectrogram_augmented)[1]
    
    # Freq Mask
    f_param = tf.cast(tf.cast(freq_bins, tf.float32) * 0.2, tf.int32)
    f = tf.random.uniform(shape=(), maxval=f_param, dtype=tf.int32)
    f0 = tf.random.uniform(shape=(), maxval=freq_bins - f, dtype=tf.int32)
    spectrogram_augmented = tf.tensor_scatter_nd_update(
        spectrogram_augmented, tf.expand_dims(tf.range(f0, f0 + f), 1), 
        tf.zeros((f, time_steps), dtype=spectrogram.dtype))
    
    # Time Mask
    t_param = tf.cast(tf.cast(time_steps, tf.float32) * 0.2, tf.int32)
    t = tf.random.uniform(shape=(), maxval=t_param, dtype=tf.int32)
    t0 = tf.random.uniform(shape=(), maxval=time_steps - t, dtype=tf.int32)
    spectrogram_augmented = tf.transpose(spectrogram_augmented)
    spectrogram_augmented = tf.tensor_scatter_nd_update(
        spectrogram_augmented, tf.expand_dims(tf.range(t0, t0 + t), 1),
        tf.zeros((t, freq_bins), dtype=spectrogram.dtype))
    spectrogram_augmented = tf.transpose(spectrogram_augmented)
    
    return tf.expand_dims(spectrogram_augmented, -1), label

def tf_spec_augment(spectrogram, label):
    [aug_spec,] = tf.py_function(spec_augment, [spectrogram, label], [spectrogram.dtype])
    aug_spec.set_shape(spectrogram.get_shape())
    return aug_spec, label

class MusicGenreEvaluator:
    """Orchestra il training, la valutazione e il salvataggio dei modelli."""
    def __init__(self, class_names):
        self.class_names = class_names
        self.results = []

    def prepare_optimizers(self, lr=1e-3):
        return {
            'Adam': optimizers.Adam(learning_rate=lr),
            'SGD_Momentum': optimizers.SGD(learning_rate=lr*10, momentum=0.9), # SGD richiede un LR più alto
            'RMSprop': optimizers.RMSprop(learning_rate=lr)
        }

    def run_experiments(self, model_factories, optimizers_config, train_data, val_data, test_data, epochs):
        for model_name, model_factory in model_factories.items():
            print(f"\n{'='*80}\nARCHITETTURA IN TEST: '{model_name}'\n{'='*80}")
            for optimizer_name, optimizer in optimizers_config.items():
                print(f"\n--- 🚀 TRAINING: [{model_name}] with [{optimizer_name}] ---")
                
                # Gestione policy per stabilità
                is_simple_cnn = 'SimpleCNN' in model_name
                original_policy = tf.keras.mixed_precision.global_policy()
                if is_simple_cnn and original_policy.name == 'mixed_float16':
                    tf.keras.mixed_precision.set_global_policy('float32')
                
                model = model_factory()
                model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
                
                callbacks_list = [
                    callbacks.EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
                    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5),
                    callbacks.ModelCheckpoint(os.path.join(MODELS_PATH, f"{model_name}_{optimizer_name}.keras"),
                                              monitor='val_accuracy', save_best_only=True, save_weights_only=False)
                ]
                
                start_time = time.time()
                history = model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=callbacks_list, verbose=2)
                training_time = time.time() - start_time
                
                print(f"--- 🧪 VALUTAZIONE su Test Set ---")
                test_loss, test_acc = model.evaluate(test_data, verbose=0)
                
                self.results.append({
                    'Model': model_name, 'Optimizer': optimizer_name,
                    'Test_Accuracy': test_acc, 'Test_Loss': test_loss,
                    'Best_Val_Accuracy': max(history.history['val_accuracy']),
                    'Epochs': len(history.history['val_accuracy']),
                    'Training_Time_s': training_time
                })
                
                if is_simple_cnn: tf.keras.mixed_precision.set_global_policy(original_policy)
        
        return pd.DataFrame(self.results)

# --- ESECUZIONE DEL TRAINING ---
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64
EPOCHS = 60

train_pipeline = tf.data.Dataset.from_tensor_slices((X_train, y_train_cat)).shuffle(len(X_train)).map(tf_spec_augment, AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_pipeline = tf.data.Dataset.from_tensor_slices((X_val, y_val_cat)).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_pipeline = tf.data.Dataset.from_tensor_slices((X_test, y_test_cat)).batch(BATCH_SIZE).prefetch(AUTOTUNE)

input_shape = X_train.shape[1:]
num_classes = y_train_cat.shape[1]

model_factories = {
    'UNet_Lite': lambda: ModelFactory.build_unet_lite_model(input_shape, num_classes),
    
    'SimpleCNN': lambda: ModelFactory.build_simple_cnn(input_shape, num_classes),
}

evaluator = MusicGenreEvaluator(class_names=label_encoder.classes_)
optimizers_config = evaluator.prepare_optimizers()
results_df = evaluator.run_experiments(
    model_factories, optimizers_config, train_pipeline, val_pipeline, test_pipeline, EPOCHS
)

# --- SALVATAGGIO DEI RISULTATI ---
results_df.to_csv(os.path.join(REPORTS_PATH, 'training_summary.csv'), index=False)
print("\n🎉 CICLO DI TRAINING COMPLETATO 🎉")
print("\nRisultati Finali:")
print(results_df)


ARCHITETTURA IN TEST: 'AudioCNN'

--- 🚀 TRAINING: [AudioCNN] with [Adam] ---
Epoch 1/60
